In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw

In [2]:
#Locate examples files
INPUT_DIR = Path(r"C:\Users\yohal\OneDrive\Programación\Python\proyectos\Excel_reporting_automation\2-Merge-Excel-Files-Real-Life\INPUT")
OUTPUT_DIR = Path(r"C:\Users\yohal\OneDrive\Programación\Python\proyectos\Excel_reporting_automation\2-Merge-Excel-Files-Real-Life")

In [4]:
#Create an empty workbook. This wb will be our final output
summary_wb = xw.Book()
summary_sht = summary_wb.sheets[0]
summary_sht.name = "Summary"

In [5]:
#Iterate over each file in directory. Copy cell range and past it into summary workbook (keep formatting & formulas)
for file in INPUT_DIR.iterdir():
    if file.suffix == ".xlsx":
        wb = xw.Book(file)
        wb.app.display_alerts = False
        #wb.app.visible = False
        sht = wb.sheets("Overview")
        values = sht.range("B5").expand().copy()
        dest_last_row = summary_sht.range("A1").expand().last_cell.row + 1
        summary_sht.range(f"A{dest_last_row}").paste()
        wb.close()


In [6]:
# Insert & style header of our summary workbook
header = [
    "Segment",
    "Country",
    "Product",
    "Discount Band",
    "Units Sold",
    "Sale Price",
    "Gross Sales",
    "Discounts",
    "Sales",
    "COGS",
    "Profit",
]
header_row = summary_sht.range("A1:K1")
header_row.value = header
header_row.font.bold = True
header_row.font.color = (255, 255, 255)
header_row.color = (119, 136, 153)

In [7]:
# Use pandas to perform data calculations
df = summary_sht.range("A1").options(pd.DataFrame, expand="table").value
data_by_country = df.groupby(by="Country").sum()[["Sales"]]
data_by_country

,Sales
Country,
Canada,1.971177e+07
France,1.922138e+07
Germany,1.727761e+07
Japan,1.619493e+07
United States of America,1.990542e+07


In [8]:
# Insert grouped dataframe into the summary workbook
summary_sht.range("M1").value = data_by_country

In [9]:
# Insert Excel chart from data_by_country

chart = summary_sht.charts.add(
    left=summary_sht.range("M8").left, 
    top=summary_sht.range("M8").top,
    width=400,
    height=200,
)
chart.set_source_data(summary_sht.range("M1").expand())
chart.chart_type = "bar_stacked"



In [10]:
# Insert PANDAS chart
ax = data_by_country.sort_values(by="Sales").plot(kind="barh")
fig = ax.get_figure()
summary_sht.pictures.add(
    fig,
    name="Sales",
    update=True,
    left=summary_sht.range("M23").left,
    top=summary_sht.range("M23").top,
    width=400,
    height=200,
)

<Picture 'Sales' in <Sheet [Libro1]Summary>>

In [11]:
# Clean up workbook, save workbook & quit Excel instance (if it is the only wb open)
summary_sht.autofit()
summary_wb.save(OUTPUT_DIR / "summary.xlsx")
if len(summary_wb.app.books) == 1:
    summary_wb.app.quit()
else:
    summary_wb.close()